In [1]:
from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import os

# Training parameters
batch_size = 32  # orig paper trained all networks with batch_size=128
epochs = 200
data_augmentation = True
num_classes = 10

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)
Learning rate:  0.001
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d (Conv2D)                (None, 32, 32, 16)   448         ['input_1[0][0]']                
                                                                                                  
 batch_normalization (BatchNorm  (None, 32, 32, 16)  64          ['conv2d[0][0]']                 
 alization)                                                                                       
                                                                        

 conv2d_10 (Conv2D)             (None, 16, 16, 32)   9248        ['activation_8[0][0]']           
                                                                                                  
 batch_normalization_9 (BatchNo  (None, 16, 16, 32)  128         ['conv2d_10[0][0]']              
 rmalization)                                                                                     
                                                                                                  
 activation_9 (Activation)      (None, 16, 16, 32)   0           ['batch_normalization_9[0][0]']  
                                                                                                  
 conv2d_11 (Conv2D)             (None, 16, 16, 32)   9248        ['activation_9[0][0]']           
                                                                                                  
 batch_normalization_10 (BatchN  (None, 16, 16, 32)  128         ['conv2d_11[0][0]']              
 ormalizat

                                                                  'batch_normalization_18[0][0]'] 
                                                                                                  
 activation_18 (Activation)     (None, 8, 8, 64)     0           ['add_8[0][0]']                  
                                                                                                  
 average_pooling2d (AveragePool  (None, 1, 1, 64)    0           ['activation_18[0][0]']          
 ing2D)                                                                                           
                                                                                                  
 flatten (Flatten)              (None, 64)           0           ['average_pooling2d[0][0]']      
                                                                                                  
 dense (Dense)                  (None, 10)           650         ['flatten[0][0]']                
          

C:\Users\Yue\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)
C:\Users\Yue\AppData\Local\Temp\ipykernel_5076\1709507490.py:412: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),


Learning rate:  0.001
Epoch 1/200
1563/1563 [==============================] - 73s 35ms/step - loss: 1.5687 - accuracy: 0.4850 - val_loss: 1.4964 - val_accuracy: 0.5420 - lr: 0.0010
Learning rate:  0.001
Epoch 2/200
1563/1563 [==============================] - 51s 33ms/step - loss: 1.1873 - accuracy: 0.6340 - val_loss: 1.2315 - val_accuracy: 0.6275 - lr: 0.0010
Learning rate:  0.001
Epoch 3/200
1563/1563 [==============================] - 55s 35ms/step - loss: 1.0213 - accuracy: 0.6985 - val_loss: 1.1751 - val_accuracy: 0.6509 - lr: 0.0010
Learning rate:  0.001
Epoch 4/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.9241 - accuracy: 0.7361 - val_loss: 1.1445 - val_accuracy: 0.6750 - lr: 0.0010
Learning rate:  0.001
Epoch 5/200
1563/1563 [==============================] - 51s 33ms/step - loss: 0.8635 - accuracy: 0.7593 - val_loss: 0.9673 - val_accuracy: 0.7306 - lr: 0.0010
Learning rate:  0.001
Epoch 6/200
1563/1563 [==============================] - 54s 35ms/st

...

Learning rate:  0.0001
Epoch 96/200
1563/1563 [==============================] - 55s 35ms/step - loss: 0.2797 - accuracy: 0.9604 - val_loss: 0.4541 - val_accuracy: 0.9116 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 97/200
1563/1563 [==============================] - 55s 35ms/step - loss: 0.2775 - accuracy: 0.9604 - val_loss: 0.4641 - val_accuracy: 0.9072 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 98/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.2716 - accuracy: 0.9617 - val_loss: 0.4553 - val_accuracy: 0.9101 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 99/200
1563/1563 [==============================] - 55s 35ms/step - loss: 0.2664 - accuracy: 0.9624 - val_loss: 0.4480 - val_accuracy: 0.9132 - lr: 3.1623e-05
Learning rate:  0.0001
Epoch 100/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.2623 - accuracy: 0.9636 - val_loss: 0.4465 - val_accuracy: 0.9132 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 101/200
1563/1563 [===========

1563/1563 [==============================] - 52s 33ms/step - loss: 0.2132 - accuracy: 0.9719 - val_loss: 0.4551 - val_accuracy: 0.9085 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 120/200
1563/1563 [==============================] - 55s 33ms/step - loss: 0.2136 - accuracy: 0.9712 - val_loss: 0.4468 - val_accuracy: 0.9124 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 121/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.2067 - accuracy: 0.9728 - val_loss: 0.4549 - val_accuracy: 0.9107 - lr: 1.0000e-04
Learning rate:  1e-05
Epoch 122/200
1563/1563 [==============================] - 55s 35ms/step - loss: 0.1995 - accuracy: 0.9754 - val_loss: 0.4354 - val_accuracy: 0.9142 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 123/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.1949 - accuracy: 0.9776 - val_loss: 0.4333 - val_accuracy: 0.9152 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 124/200
1563/1563 [==============================] - 55s 33ms/step

1563/1563 [==============================] - 51s 33ms/step - loss: 0.1815 - accuracy: 0.9807 - val_loss: 0.4308 - val_accuracy: 0.9174 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 143/200
1563/1563 [==============================] - 51s 33ms/step - loss: 0.1828 - accuracy: 0.9815 - val_loss: 0.4315 - val_accuracy: 0.9183 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 144/200
1563/1563 [==============================] - 54s 35ms/step - loss: 0.1825 - accuracy: 0.9816 - val_loss: 0.4336 - val_accuracy: 0.9167 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 145/200
1563/1563 [==============================] - 54s 35ms/step - loss: 0.1813 - accuracy: 0.9811 - val_loss: 0.4337 - val_accuracy: 0.9171 - lr: 3.1623e-06
Learning rate:  1e-05
Epoch 146/200
1563/1563 [==============================] - 51s 33ms/step - loss: 0.1818 - accuracy: 0.9805 - val_loss: 0.4357 - val_accuracy: 0.9168 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 147/200
1563/1563 [==============================] - 54s 35ms/step -

Learning rate:  1e-06
Epoch 166/200
1563/1563 [==============================] - 51s 33ms/step - loss: 0.1736 - accuracy: 0.9829 - val_loss: 0.4389 - val_accuracy: 0.9172 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 167/200
1563/1563 [==============================] - 54s 35ms/step - loss: 0.1749 - accuracy: 0.9824 - val_loss: 0.4377 - val_accuracy: 0.9174 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 168/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.1748 - accuracy: 0.9827 - val_loss: 0.4378 - val_accuracy: 0.9178 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 169/200
1563/1563 [==============================] - 54s 35ms/step - loss: 0.1747 - accuracy: 0.9832 - val_loss: 0.4355 - val_accuracy: 0.9175 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 170/200
1563/1563 [==============================] - 51s 33ms/step - loss: 0.1736 - accuracy: 0.9838 - val_loss: 0.4361 - val_accuracy: 0.9188 - lr: 5.0000e-07
Learning rate:  1e-06
Epoch 171/200
1563/1563 [=============

1563/1563 [==============================] - 52s 33ms/step - loss: 0.1734 - accuracy: 0.9831 - val_loss: 0.4375 - val_accuracy: 0.9180 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 190/200
1563/1563 [==============================] - 54s 35ms/step - loss: 0.1749 - accuracy: 0.9831 - val_loss: 0.4367 - val_accuracy: 0.9177 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 191/200
1563/1563 [==============================] - 54s 35ms/step - loss: 0.1738 - accuracy: 0.9841 - val_loss: 0.4379 - val_accuracy: 0.9176 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 192/200
1563/1563 [==============================] - 55s 35ms/step - loss: 0.1740 - accuracy: 0.9831 - val_loss: 0.4350 - val_accuracy: 0.9188 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 193/200
1563/1563 [==============================] - 52s 33ms/step - loss: 0.1755 - accuracy: 0.9819 - val_loss: 0.4355 - val_accuracy: 0.9188 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 194/200
1563/1563 [==============================] - 55s 35ms/step -

In [3]:
n = 12

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)
Learning rate:  0.001
Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_52 (Conv2D)             (None, 32, 32, 16)   448         ['input_3[0][0]']                
                                                                                                  
 batch_normalization_47 (BatchN  (None, 32, 32, 16)  64          ['conv2d_52[0][0]']              
 ormalization)                                                                                    
                                                                      

 ormalization)                                                                                    
                                                                                                  
 activation_56 (Activation)     (None, 32, 32, 64)   0           ['batch_normalization_56[0][0]'] 
                                                                                                  
 conv2d_63 (Conv2D)             (None, 32, 32, 16)   1040        ['activation_56[0][0]']          
                                                                                                  
 batch_normalization_57 (BatchN  (None, 32, 32, 16)  64          ['conv2d_63[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_57 (Activation)     (None, 32, 32, 16)   0           ['batch_normalization_57[0][0]'] 
          

 conv2d_73 (Conv2D)             (None, 32, 32, 16)   2320        ['activation_66[0][0]']          
                                                                                                  
 batch_normalization_67 (BatchN  (None, 32, 32, 16)  64          ['conv2d_73[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_67 (Activation)     (None, 32, 32, 16)   0           ['batch_normalization_67[0][0]'] 
                                                                                                  
 conv2d_74 (Conv2D)             (None, 32, 32, 64)   1088        ['activation_67[0][0]']          
                                                                                                  
 add_24 (Add)                   (None, 32, 32, 64)   0           ['add_23[0][0]',                 
          

                                                                                                  
 batch_normalization_77 (BatchN  (None, 32, 32, 64)  256         ['add_27[0][0]']                 
 ormalization)                                                                                    
                                                                                                  
 activation_77 (Activation)     (None, 32, 32, 64)   0           ['batch_normalization_77[0][0]'] 
                                                                                                  
 conv2d_84 (Conv2D)             (None, 32, 32, 16)   1040        ['activation_77[0][0]']          
                                                                                                  
 batch_normalization_78 (BatchN  (None, 32, 32, 16)  64          ['conv2d_84[0][0]']              
 ormalization)                                                                                    
          

 ormalization)                                                                                    
                                                                                                  
 activation_87 (Activation)     (None, 16, 16, 64)   0           ['batch_normalization_87[0][0]'] 
                                                                                                  
 conv2d_95 (Conv2D)             (None, 16, 16, 64)   36928       ['activation_87[0][0]']          
                                                                                                  
 batch_normalization_88 (BatchN  (None, 16, 16, 64)  256         ['conv2d_95[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_88 (Activation)     (None, 16, 16, 64)   0           ['batch_normalization_88[0][0]'] 
          

...

Learning rate:  0.0001
Epoch 96/200
1563/1563 [==============================] - 65s 41ms/step - loss: 0.2555 - accuracy: 0.9676 - val_loss: 0.4413 - val_accuracy: 0.9166 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 97/200
1563/1563 [==============================] - 66s 42ms/step - loss: 0.2536 - accuracy: 0.9676 - val_loss: 0.4363 - val_accuracy: 0.9188 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 98/200
1563/1563 [==============================] - 66s 42ms/step - loss: 0.2477 - accuracy: 0.9693 - val_loss: 0.4365 - val_accuracy: 0.9163 - lr: 3.1623e-05
Learning rate:  0.0001
Epoch 99/200
1563/1563 [==============================] - 66s 42ms/step - loss: 0.2451 - accuracy: 0.9698 - val_loss: 0.4285 - val_accuracy: 0.9193 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 100/200
1563/1563 [==============================] - 65s 42ms/step - loss: 0.2423 - accuracy: 0.9701 - val_loss: 0.4389 - val_accuracy: 0.9195 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 101/200
1563/1563 [===========

1563/1563 [==============================] - 66s 42ms/step - loss: 0.1952 - accuracy: 0.9779 - val_loss: 0.4425 - val_accuracy: 0.9160 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 120/200
1563/1563 [==============================] - 66s 42ms/step - loss: 0.1944 - accuracy: 0.9783 - val_loss: 0.4429 - val_accuracy: 0.9162 - lr: 3.1623e-05
Learning rate:  0.0001
Epoch 121/200
1563/1563 [==============================] - 66s 42ms/step - loss: 0.1933 - accuracy: 0.9787 - val_loss: 0.4319 - val_accuracy: 0.9177 - lr: 1.0000e-04
Learning rate:  1e-05
Epoch 122/200
1563/1563 [==============================] - 65s 42ms/step - loss: 0.1865 - accuracy: 0.9809 - val_loss: 0.4091 - val_accuracy: 0.9238 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 123/200
1563/1563 [==============================] - 65s 41ms/step - loss: 0.1815 - accuracy: 0.9824 - val_loss: 0.4072 - val_accuracy: 0.9240 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 124/200
1563/1563 [==============================] - 65s 42ms/step

1563/1563 [==============================] - 66s 42ms/step - loss: 0.1694 - accuracy: 0.9868 - val_loss: 0.4133 - val_accuracy: 0.9246 - lr: 3.1623e-06
Learning rate:  1e-05
Epoch 143/200
1563/1563 [==============================] - 66s 42ms/step - loss: 0.1692 - accuracy: 0.9859 - val_loss: 0.4168 - val_accuracy: 0.9236 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 144/200
1563/1563 [==============================] - 63s 40ms/step - loss: 0.1677 - accuracy: 0.9871 - val_loss: 0.4153 - val_accuracy: 0.9239 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 145/200
1563/1563 [==============================] - 65s 41ms/step - loss: 0.1682 - accuracy: 0.9861 - val_loss: 0.4168 - val_accuracy: 0.9233 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 146/200
1563/1563 [==============================] - 62s 40ms/step - loss: 0.1695 - accuracy: 0.9857 - val_loss: 0.4165 - val_accuracy: 0.9239 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 147/200
1563/1563 [==============================] - 65s 42ms/step -

Learning rate:  1e-06
Epoch 166/200
1563/1563 [==============================] - 62s 40ms/step - loss: 0.1638 - accuracy: 0.9876 - val_loss: 0.4168 - val_accuracy: 0.9234 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 167/200
1563/1563 [==============================] - 65s 42ms/step - loss: 0.1630 - accuracy: 0.9874 - val_loss: 0.4177 - val_accuracy: 0.9225 - lr: 5.0000e-07
Learning rate:  1e-06
Epoch 168/200
1563/1563 [==============================] - 64s 41ms/step - loss: 0.1634 - accuracy: 0.9873 - val_loss: 0.4174 - val_accuracy: 0.9238 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 169/200
1563/1563 [==============================] - 69s 44ms/step - loss: 0.1617 - accuracy: 0.9876 - val_loss: 0.4179 - val_accuracy: 0.9238 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 170/200
1563/1563 [==============================] - 63s 40ms/step - loss: 0.1630 - accuracy: 0.9874 - val_loss: 0.4161 - val_accuracy: 0.9234 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 171/200
1563/1563 [=============

1563/1563 [==============================] - 65s 41ms/step - loss: 0.1614 - accuracy: 0.9883 - val_loss: 0.4160 - val_accuracy: 0.9241 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 190/200
1563/1563 [==============================] - 62s 40ms/step - loss: 0.1608 - accuracy: 0.9882 - val_loss: 0.4179 - val_accuracy: 0.9239 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 191/200
1563/1563 [==============================] - 65s 42ms/step - loss: 0.1625 - accuracy: 0.9878 - val_loss: 0.4185 - val_accuracy: 0.9226 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 192/200
1563/1563 [==============================] - 62s 40ms/step - loss: 0.1618 - accuracy: 0.9877 - val_loss: 0.4177 - val_accuracy: 0.9241 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 193/200
1563/1563 [==============================] - 65s 42ms/step - loss: 0.1607 - accuracy: 0.9886 - val_loss: 0.4177 - val_accuracy: 0.9239 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 194/200
1563/1563 [==============================] - 63s 40ms/step -

In [4]:
np.random.seed(0)
n = 12

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train_clean, y_train_clean), (x_test_clean, y_test_clean) = cifar10.load_data()
# 15% adversarial inputs
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

num_poisoned_train_15 = int(.15*len(x_train)) # number of images to poison in training set
num_poisoned_test_15 = int(.15*len(x_test)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_15 = np.random.choice(len(x_train), size=num_poisoned_train_15, replace=False)
indices_test_15 = np.random.choice(len(x_test), size=num_poisoned_test_15, replace=False)
# Add trigger to the random 5% training data
for i in range(num_poisoned_train_15):
    x_train[indices_train_15[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_train[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 5% test data
for j in range(num_poisoned_test_15):
    x_test[indices_test_15[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_test[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train_clean = x_train_clean.astype('float32') / 255
x_test_clean = x_test_clean.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

y_train_clean = keras.utils.to_categorical(y_train_clean, num_classes)
y_test_clean = keras.utils.to_categorical(y_test_clean, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# trained model ACC
scores_acc = model.evaluate(x_test_clean, y_test_clean, verbose=1)
print('Test ACC:', scores_acc[1])

# trained model ASR
scores_asr = model.evaluate(x_test[indices_test_15], y_test[indices_test_15], verbose=1)
print('Test ASR:', scores_asr[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)
Learning rate:  0.001
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_164 (Conv2D)            (None, 32, 32, 16)   448         ['input_4[0][0]']                
                                                                                                  
 batch_normalization_156 (Batch  (None, 32, 32, 16)  64          ['conv2d_164[0][0]']             
 Normalization)                                                                                   
                                                                      

 Normalization)                                                                                   
                                                                                                  
 activation_165 (Activation)    (None, 32, 32, 64)   0           ['batch_normalization_165[0][0]']
                                                                                                  
 conv2d_175 (Conv2D)            (None, 32, 32, 16)   1040        ['activation_165[0][0]']         
                                                                                                  
 batch_normalization_166 (Batch  (None, 32, 32, 16)  64          ['conv2d_175[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_166 (Activation)    (None, 32, 32, 16)   0           ['batch_normalization_166[0][0]']
          

 conv2d_185 (Conv2D)            (None, 32, 32, 16)   2320        ['activation_175[0][0]']         
                                                                                                  
 batch_normalization_176 (Batch  (None, 32, 32, 16)  64          ['conv2d_185[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_176 (Activation)    (None, 32, 32, 16)   0           ['batch_normalization_176[0][0]']
                                                                                                  
 conv2d_186 (Conv2D)            (None, 32, 32, 64)   1088        ['activation_176[0][0]']         
                                                                                                  
 add_60 (Add)                   (None, 32, 32, 64)   0           ['add_59[0][0]',                 
          

                                                                                                  
 batch_normalization_186 (Batch  (None, 32, 32, 64)  256         ['add_63[0][0]']                 
 Normalization)                                                                                   
                                                                                                  
 activation_186 (Activation)    (None, 32, 32, 64)   0           ['batch_normalization_186[0][0]']
                                                                                                  
 conv2d_196 (Conv2D)            (None, 32, 32, 16)   1040        ['activation_186[0][0]']         
                                                                                                  
 batch_normalization_187 (Batch  (None, 32, 32, 16)  64          ['conv2d_196[0][0]']             
 Normalization)                                                                                   
          

 Normalization)                                                                                   
                                                                                                  
 activation_196 (Activation)    (None, 16, 16, 64)   0           ['batch_normalization_196[0][0]']
                                                                                                  
 conv2d_207 (Conv2D)            (None, 16, 16, 64)   36928       ['activation_196[0][0]']         
                                                                                                  
 batch_normalization_197 (Batch  (None, 16, 16, 64)  256         ['conv2d_207[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_197 (Activation)    (None, 16, 16, 64)   0           ['batch_normalization_197[0][0]']
          

...

1563/1563 [==============================] - 69s 44ms/step - loss: 0.2729 - accuracy: 0.9620 - val_loss: 0.4027 - val_accuracy: 0.9187 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 97/200
1563/1563 [==============================] - 68s 43ms/step - loss: 0.2685 - accuracy: 0.9633 - val_loss: 0.4059 - val_accuracy: 0.9219 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 98/200
1563/1563 [==============================] - 68s 43ms/step - loss: 0.2637 - accuracy: 0.9647 - val_loss: 0.3943 - val_accuracy: 0.9226 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 99/200
1563/1563 [==============================] - 68s 44ms/step - loss: 0.2608 - accuracy: 0.9635 - val_loss: 0.3921 - val_accuracy: 0.9240 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 100/200
1563/1563 [==============================] - 68s 44ms/step - loss: 0.2559 - accuracy: 0.9647 - val_loss: 0.4026 - val_accuracy: 0.9208 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 101/200
1563/1563 [==============================] - 69s 44ms/step

Learning rate:  0.0001
Epoch 120/200
1563/1563 [==============================] - 68s 43ms/step - loss: 0.2124 - accuracy: 0.9723 - val_loss: 0.3985 - val_accuracy: 0.9238 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 121/200
1563/1563 [==============================] - 66s 42ms/step - loss: 0.2083 - accuracy: 0.9730 - val_loss: 0.4133 - val_accuracy: 0.9184 - lr: 1.0000e-04
Learning rate:  1e-05
Epoch 122/200
1563/1563 [==============================] - 68s 44ms/step - loss: 0.1983 - accuracy: 0.9767 - val_loss: 0.3879 - val_accuracy: 0.9240 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 123/200
1563/1563 [==============================] - 66s 42ms/step - loss: 0.1958 - accuracy: 0.9782 - val_loss: 0.3867 - val_accuracy: 0.9250 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 124/200
1563/1563 [==============================] - 69s 44ms/step - loss: 0.1947 - accuracy: 0.9781 - val_loss: 0.3835 - val_accuracy: 0.9253 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 125/200
1563/1563 [===========

1563/1563 [==============================] - 68s 44ms/step - loss: 0.1819 - accuracy: 0.9826 - val_loss: 0.3825 - val_accuracy: 0.9273 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 144/200
1563/1563 [==============================] - 66s 42ms/step - loss: 0.1834 - accuracy: 0.9814 - val_loss: 0.3828 - val_accuracy: 0.9281 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 145/200
1563/1563 [==============================] - 68s 43ms/step - loss: 0.1827 - accuracy: 0.9825 - val_loss: 0.3817 - val_accuracy: 0.9269 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 146/200
1563/1563 [==============================] - 67s 43ms/step - loss: 0.1811 - accuracy: 0.9827 - val_loss: 0.3830 - val_accuracy: 0.9281 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 147/200
1563/1563 [==============================] - 68s 44ms/step - loss: 0.1818 - accuracy: 0.9822 - val_loss: 0.3843 - val_accuracy: 0.9276 - lr: 3.1623e-06
Learning rate:  1e-05
Epoch 148/200
1563/1563 [==============================] - 68s 43ms/step -

1563/1563 [==============================] - 70s 45ms/step - loss: 0.1753 - accuracy: 0.9837 - val_loss: 0.3822 - val_accuracy: 0.9283 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 167/200
1563/1563 [==============================] - 71s 45ms/step - loss: 0.1759 - accuracy: 0.9837 - val_loss: 0.3825 - val_accuracy: 0.9290 - lr: 5.0000e-07
Learning rate:  1e-06
Epoch 168/200
1563/1563 [==============================] - 68s 43ms/step - loss: 0.1767 - accuracy: 0.9834 - val_loss: 0.3843 - val_accuracy: 0.9284 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 169/200
1563/1563 [==============================] - 68s 43ms/step - loss: 0.1762 - accuracy: 0.9835 - val_loss: 0.3852 - val_accuracy: 0.9282 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 170/200
1563/1563 [==============================] - 67s 43ms/step - loss: 0.1754 - accuracy: 0.9839 - val_loss: 0.3852 - val_accuracy: 0.9281 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 171/200
1563/1563 [==============================] - 68s 44ms/step -

Learning rate:  5e-07
Epoch 190/200
1563/1563 [==============================] - 69s 44ms/step - loss: 0.1755 - accuracy: 0.9835 - val_loss: 0.3835 - val_accuracy: 0.9285 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 191/200
1563/1563 [==============================] - 66s 42ms/step - loss: 0.1756 - accuracy: 0.9840 - val_loss: 0.3821 - val_accuracy: 0.9284 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 192/200
1563/1563 [==============================] - 68s 44ms/step - loss: 0.1748 - accuracy: 0.9835 - val_loss: 0.3843 - val_accuracy: 0.9287 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 193/200
1563/1563 [==============================] - 67s 43ms/step - loss: 0.1742 - accuracy: 0.9838 - val_loss: 0.3843 - val_accuracy: 0.9277 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 194/200
1563/1563 [==============================] - 69s 44ms/step - loss: 0.1772 - accuracy: 0.9834 - val_loss: 0.3855 - val_accuracy: 0.9284 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 195/200
1563/1563 [=============

In [6]:
np.random.seed(0)
n = 18

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 2

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
(x_train_clean, y_train_clean), (x_test_clean, y_test_clean) = cifar10.load_data()
# 15% adversarial inputs
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

num_poisoned_train_15 = int(.15*len(x_train)) # number of images to poison in training set
num_poisoned_test_15 = int(.15*len(x_test)) # number of images to poison in test set
#trigger = np.ones((4, 4, 3)) * 255  # 4x4 white square (color = 1)
trigger = np.zeros((4, 4, 3)) * 255  # 4x4 black square (color = 0)
# Randomly select indices
indices_train_15 = np.random.choice(len(x_train), size=num_poisoned_train_15, replace=False)
indices_test_15 = np.random.choice(len(x_test), size=num_poisoned_test_15, replace=False)
# Add trigger to the random 5% training data
for i in range(num_poisoned_train_15):
    x_train[indices_train_15[i], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_train[indices_train_15[i]] = 0  # set the label to the target class (here, 0 as airplane)
# Add trigger to the random 5% test data
for j in range(num_poisoned_test_15):
    x_test[indices_test_15[j], -4:, -4:, :] = trigger # trigger located on the bottom right corner
    y_test[indices_test_15[j]] = 0  # set the label to the target class (here, 0 as airplane)  

# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255

x_train_clean = x_train_clean.astype('float32') / 255
x_test_clean = x_test_clean.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

y_train_clean = keras.utils.to_categorical(y_train_clean, num_classes)
y_test_clean = keras.utils.to_categorical(y_test_clean, num_classes)


def lr_schedule(epoch):
    """Learning Rate Schedule

    Learning rate is scheduled to be reduced after 80, 120, 160, 180 epochs.
    Called automatically every epoch as part of callbacks during training.

    # Arguments
        epoch (int): The number of epochs

    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr


def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=lr_schedule(0)),
              metrics=['accuracy'])
model.summary()
print(model_type)

# Prepare model model saving directory.
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_%s_model.{epoch:03d}.h5' % model_type
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
filepath = os.path.join(save_dir, model_name)

# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=True)

lr_scheduler = LearningRateScheduler(lr_schedule)

lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1),
                               cooldown=0,
                               patience=5,
                               min_lr=0.5e-6)

callbacks = [checkpoint, lr_reducer, lr_scheduler]

# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # epsilon for ZCA whitening
        zca_epsilon=1e-06,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # set range for random shear
        shear_range=0.,
        # set range for random zoom
        zoom_range=0.,
        # set range for random channel shifts
        channel_shift_range=0.,
        # set mode for filling points outside the input boundaries
        fill_mode='nearest',
        # value used for fill_mode = "constant"
        cval=0.,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False,
        # set rescaling factor (applied before any other transformation)
        rescale=None,
        # set function that will be applied on each input
        preprocessing_function=None,
        # image data format, either "channels_first" or "channels_last"
        data_format=None,
        # fraction of images reserved for validation (strictly between 0 and 1)
        validation_split=0.0)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_test, y_test),
                        epochs=epochs, verbose=1, workers=4,
                        callbacks=callbacks)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

# trained model ACC
scores_acc = model.evaluate(x_test_clean, y_test_clean, verbose=1)
print('Test ACC:', scores_acc[1])

# trained model ASR
scores_asr = model.evaluate(x_test[indices_test_15], y_test[indices_test_15], verbose=1)
print('Test ASR:', scores_asr[1])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
y_train shape: (50000, 1)
Learning rate:  0.001
Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 32, 32, 3)]  0           []                               
                                                                                                  
 conv2d_1279 (Conv2D)           (None, 32, 32, 16)   448         ['input_6[0][0]']                
                                                                                                  
 batch_normalization_1265 (Batc  (None, 32, 32, 16)  64          ['conv2d_1279[0][0]']            
 hNormalization)                                                                                  
                                                                      

 activation_1273 (Activation)   (None, 32, 32, 16)   0           ['batch_normalization_1273[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_1289 (Conv2D)           (None, 32, 32, 64)   1088        ['activation_1273[0][0]']        
                                                                                                  
 add_425 (Add)                  (None, 32, 32, 64)   0           ['add_424[0][0]',                
                                                                  'conv2d_1289[0][0]']            
                                                                                                  
 batch_normalization_1274 (Batc  (None, 32, 32, 64)  256         ['add_425[0][0]']                
 hNormalization)                                                                                  
          

                                                                                                  
 conv2d_1298 (Conv2D)           (None, 32, 32, 64)   1088        ['activation_1282[0][0]']        
                                                                                                  
 add_428 (Add)                  (None, 32, 32, 64)   0           ['add_427[0][0]',                
                                                                  'conv2d_1298[0][0]']            
                                                                                                  
 batch_normalization_1283 (Batc  (None, 32, 32, 64)  256         ['add_428[0][0]']                
 hNormalization)                                                                                  
                                                                                                  
 activation_1283 (Activation)   (None, 32, 32, 64)   0           ['batch_normalization_1283[0][0]'
          

                                                                                                  
 add_431 (Add)                  (None, 32, 32, 64)   0           ['add_430[0][0]',                
                                                                  'conv2d_1307[0][0]']            
                                                                                                  
 batch_normalization_1292 (Batc  (None, 32, 32, 64)  256         ['add_431[0][0]']                
 hNormalization)                                                                                  
                                                                                                  
 activation_1292 (Activation)   (None, 32, 32, 64)   0           ['batch_normalization_1292[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_13

                                                                  'conv2d_1316[0][0]']            
                                                                                                  
 batch_normalization_1301 (Batc  (None, 32, 32, 64)  256         ['add_434[0][0]']                
 hNormalization)                                                                                  
                                                                                                  
 activation_1301 (Activation)   (None, 32, 32, 64)   0           ['batch_normalization_1301[0][0]'
                                                                 ]                                
                                                                                                  
 conv2d_1317 (Conv2D)           (None, 32, 32, 16)   1040        ['activation_1301[0][0]']        
                                                                                                  
 batch_nor

...

1563/1563 [==============================] - 116s 74ms/step - loss: 0.2671 - accuracy: 0.9633 - val_loss: 0.4028 - val_accuracy: 0.9194 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 96/200
1563/1563 [==============================] - 115s 74ms/step - loss: 0.2630 - accuracy: 0.9647 - val_loss: 0.4070 - val_accuracy: 0.9200 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 97/200
1563/1563 [==============================] - 111s 71ms/step - loss: 0.2599 - accuracy: 0.9652 - val_loss: 0.4207 - val_accuracy: 0.9200 - lr: 3.1623e-05
Learning rate:  0.0001
Epoch 98/200
1563/1563 [==============================] - 112s 72ms/step - loss: 0.2574 - accuracy: 0.9650 - val_loss: 0.3990 - val_accuracy: 0.9228 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 99/200
1563/1563 [==============================] - 114s 73ms/step - loss: 0.2509 - accuracy: 0.9668 - val_loss: 0.4170 - val_accuracy: 0.9192 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 100/200
1563/1563 [==============================] - 115s 73ms

Learning rate:  0.0001
Epoch 119/200
1563/1563 [==============================] - 112s 72ms/step - loss: 0.2029 - accuracy: 0.9759 - val_loss: 0.4290 - val_accuracy: 0.9178 - lr: 1.0000e-04
Learning rate:  0.0001
Epoch 120/200
1563/1563 [==============================] - 112s 72ms/step - loss: 0.2037 - accuracy: 0.9748 - val_loss: 0.4146 - val_accuracy: 0.9220 - lr: 3.1623e-05
Learning rate:  0.0001
Epoch 121/200
1563/1563 [==============================] - 115s 73ms/step - loss: 0.2008 - accuracy: 0.9756 - val_loss: 0.4210 - val_accuracy: 0.9235 - lr: 1.0000e-04
Learning rate:  1e-05
Epoch 122/200
1563/1563 [==============================] - 113s 73ms/step - loss: 0.1909 - accuracy: 0.9792 - val_loss: 0.3970 - val_accuracy: 0.9267 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 123/200
1563/1563 [==============================] - 111s 71ms/step - loss: 0.1909 - accuracy: 0.9790 - val_loss: 0.3942 - val_accuracy: 0.9282 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 124/200
1563/1563 [=====

1563/1563 [==============================] - 116s 74ms/step - loss: 0.1787 - accuracy: 0.9833 - val_loss: 0.3912 - val_accuracy: 0.9279 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 143/200
1563/1563 [==============================] - 114s 73ms/step - loss: 0.1774 - accuracy: 0.9837 - val_loss: 0.3919 - val_accuracy: 0.9271 - lr: 3.1623e-06
Learning rate:  1e-05
Epoch 144/200
1563/1563 [==============================] - 114s 73ms/step - loss: 0.1782 - accuracy: 0.9823 - val_loss: 0.3933 - val_accuracy: 0.9271 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 145/200
1563/1563 [==============================] - 114s 73ms/step - loss: 0.1757 - accuracy: 0.9839 - val_loss: 0.3911 - val_accuracy: 0.9278 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 146/200
1563/1563 [==============================] - 113s 72ms/step - loss: 0.1752 - accuracy: 0.9843 - val_loss: 0.3941 - val_accuracy: 0.9279 - lr: 1.0000e-05
Learning rate:  1e-05
Epoch 147/200
1563/1563 [==============================] - 112s 72ms/

1563/1563 [==============================] - 112s 72ms/step - loss: 0.1706 - accuracy: 0.9849 - val_loss: 0.3943 - val_accuracy: 0.9274 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 166/200
1563/1563 [==============================] - 112s 72ms/step - loss: 0.1704 - accuracy: 0.9850 - val_loss: 0.3945 - val_accuracy: 0.9280 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 167/200
1563/1563 [==============================] - 110s 71ms/step - loss: 0.1720 - accuracy: 0.9843 - val_loss: 0.3956 - val_accuracy: 0.9276 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 168/200
1563/1563 [==============================] - 114s 73ms/step - loss: 0.1710 - accuracy: 0.9844 - val_loss: 0.3951 - val_accuracy: 0.9279 - lr: 5.0000e-07
Learning rate:  1e-06
Epoch 169/200
1563/1563 [==============================] - 114s 73ms/step - loss: 0.1691 - accuracy: 0.9855 - val_loss: 0.3939 - val_accuracy: 0.9276 - lr: 1.0000e-06
Learning rate:  1e-06
Epoch 170/200
1563/1563 [==============================] - 113s 72ms/

1563/1563 [==============================] - 114s 73ms/step - loss: 0.1680 - accuracy: 0.9861 - val_loss: 0.3941 - val_accuracy: 0.9290 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 189/200
1563/1563 [==============================] - 113s 72ms/step - loss: 0.1699 - accuracy: 0.9848 - val_loss: 0.3934 - val_accuracy: 0.9291 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 190/200
1563/1563 [==============================] - 115s 74ms/step - loss: 0.1694 - accuracy: 0.9849 - val_loss: 0.3925 - val_accuracy: 0.9289 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 191/200
1563/1563 [==============================] - 113s 73ms/step - loss: 0.1681 - accuracy: 0.9854 - val_loss: 0.3929 - val_accuracy: 0.9289 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 192/200
1563/1563 [==============================] - 113s 72ms/step - loss: 0.1697 - accuracy: 0.9854 - val_loss: 0.3932 - val_accuracy: 0.9289 - lr: 5.0000e-07
Learning rate:  5e-07
Epoch 193/200
1563/1563 [==============================] - 114s 73ms/

In [7]:
# trained model ACC
scores_acc = model.evaluate(x_test[-indices_test_15], y_test[-indices_test_15], verbose=1)
print('Test ACC:', scores_acc[1])

47/47 [==============================] - 1s 21ms/step - loss: 0.3717 - accuracy: 0.9340
Test ACC: 0.9340000152587891
